In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
file_paths = {
    "Newsletter Interaction": "Newsletter_Interaction_Data.csv",
    "Product Bug Task": "Product_Bug_Task_Data.csv",
    "Region and Vertical": "RegionAndVertical_Data.csv",
    "Status and Level": "StatusAndLevel_Data.csv",
    "CSAT Survey": "CSAT_Survey_Data.csv",
    "Customer Age": "Customer_Age_Data.csv",
    "Customer MRR": "Customer_MRR_Data.csv",
    "Customer Revenue": "Customer_Revenue_Data.csv",
    "Help Ticket": "Help_Ticket_Data.csv",
}

dict_items([('Newsletter Interaction', 'Newsletter_Interaction_Data.csv'), ('Product Bug Task', 'Product_Bug_Task_Data.csv'), ('Region and Vertical', 'RegionAndVertical_Data.csv'), ('Status and Level', 'StatusAndLevel_Data.csv'), ('CSAT Survey', 'CSAT_Survey_Data.csv'), ('Customer Age', 'Customer_Age_Data.csv'), ('Customer MRR', 'Customer_MRR_Data.csv'), ('Customer Revenue', 'Customer_Revenue_Data.csv'), ('Help Ticket', 'Help_Ticket_Data.csv')])

In [10]:
dataframes = {name: pd.read_csv(f"archive/{path}") for name, path in file_paths.items()}

In [13]:
dataframes["Customer Age"].rename(columns={"CRM ID": "Customer ID"}, inplace=True)

In [29]:
def clean_currency(value):
    if isinstance(value, str):
        return float(value.replace('$', '').replace(',', ''))
    return value

dataframes["Customer MRR"]["MRR"] = dataframes["Customer MRR"]["MRR"].apply(clean_currency)
dataframes["Customer Revenue"]["Total Revenue"] = dataframes["Customer Revenue"]["Total Revenue"].apply(clean_currency)

In [25]:
for name, df in dataframes.items():
    print(f"📌 {name} veri setinde eşsiz müşteri sayısı:", df["Customer ID"].nunique())
    print(f"➡️ {name} veri setinde toplam satır sayısı:", len(df))

📌 Newsletter Interaction veri setinde eşsiz müşteri sayısı: 251
➡️ Newsletter Interaction veri setinde toplam satır sayısı: 251
📌 Product Bug Task veri setinde eşsiz müşteri sayısı: 640
➡️ Product Bug Task veri setinde toplam satır sayısı: 1989
📌 Region and Vertical veri setinde eşsiz müşteri sayısı: 2651
➡️ Region and Vertical veri setinde toplam satır sayısı: 2655
📌 Status and Level veri setinde eşsiz müşteri sayısı: 2651
➡️ Status and Level veri setinde toplam satır sayısı: 2655
📌 CSAT Survey veri setinde eşsiz müşteri sayısı: 640
➡️ CSAT Survey veri setinde toplam satır sayısı: 1989
📌 Customer Age veri setinde eşsiz müşteri sayısı: 2651
➡️ Customer Age veri setinde toplam satır sayısı: 2651
📌 Customer MRR veri setinde eşsiz müşteri sayısı: 1380
➡️ Customer MRR veri setinde toplam satır sayısı: 1380
📌 Customer Revenue veri setinde eşsiz müşteri sayısı: 901
➡️ Customer Revenue veri setinde toplam satır sayısı: 901
📌 Help Ticket veri setinde eşsiz müşteri sayısı: 1367
➡️ Help Ticket v

In [32]:
print(dataframes["CSAT Survey"].columns)

Index(['Customer ID',
       'How likely are you to recommend insider to a friend or colleague ',
       'How would you rate the value you gain from our company',
       'Please rate the overall quality of our products',
       'Please rate the usability of the panel'],
      dtype='object')


In [34]:
dataframes["Product Bug Task"] = dataframes["Product Bug Task"].groupby("Customer ID", as_index=False)["Product Bug Task Count"].sum()

csat_columns = [
    "How likely are you to recommend insider to a friend or colleague ",
    "How would you rate the value you gain from our company",
    "Please rate the overall quality of our products",
    "Please rate the usability of the panel"
]
dataframes["CSAT Survey"] = dataframes["CSAT Survey"].groupby("Customer ID", as_index=False)[csat_columns].mean()

dataframes["Region and Vertical"] = dataframes["Region and Vertical"].groupby("Customer ID").agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]).reset_index()

dataframes["Status and Level"] = dataframes["Status and Level"].groupby("Customer ID").agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]).reset_index()

In [35]:
merged_df = dataframes["Customer Age"]

for key, df in dataframes.items():
    if key != "Customer Age":  # Zaten başlangıç noktası olarak kullanıyoruz
        merged_df = merged_df.merge(df, on="Customer ID", how="left")

In [40]:
print("Merged Satır Sayısı:", len(merged_df))

print("Unique Customer ID Sayısı:", merged_df["Customer ID"].nunique())

for name, df in dataframes.items():
    print(f"{name} veri setinde eşsiz müşteri sayısı:", df["Customer ID"].nunique())
    print(f"{name} veri setinde toplam satır sayısı:", len(df))


Merged Satır Sayısı: 2651
Unique Customer ID Sayısı: 2651
Newsletter Interaction veri setinde eşsiz müşteri sayısı: 251
Newsletter Interaction veri setinde toplam satır sayısı: 251
Product Bug Task veri setinde eşsiz müşteri sayısı: 640
Product Bug Task veri setinde toplam satır sayısı: 640
Region and Vertical veri setinde eşsiz müşteri sayısı: 2651
Region and Vertical veri setinde toplam satır sayısı: 2651
Status and Level veri setinde eşsiz müşteri sayısı: 2651
Status and Level veri setinde toplam satır sayısı: 2651
CSAT Survey veri setinde eşsiz müşteri sayısı: 640
CSAT Survey veri setinde toplam satır sayısı: 640
Customer Age veri setinde eşsiz müşteri sayısı: 2651
Customer Age veri setinde toplam satır sayısı: 2651
Customer MRR veri setinde eşsiz müşteri sayısı: 1380
Customer MRR veri setinde toplam satır sayısı: 1380
Customer Revenue veri setinde eşsiz müşteri sayısı: 901
Customer Revenue veri setinde toplam satır sayısı: 901
Help Ticket veri setinde eşsiz müşteri sayısı: 1367
He

In [ ]:
missing_values = merged_df.isnull().sum()
missing_percentage = (missing_values / len(merged_df)) * 100

missing_df = pd.DataFrame({"Eksik Değer Sayısı": missing_values, "Eksik Veri (%)": missing_percentage})
missing_df = missing_df[missing_df["Eksik Değer Sayısı"] > 0]  

print("\nEksik Veri Analizi:")
print(missing_df)



Eksik Veri Analizi:
                                                    Eksik Değer Sayısı  \
Company Newsletter Interaction Count                              2400   
Product Bug Task Count                                            2013   
Region                                                               1   
Vertical                                                            66   
Subvertical                                                        102   
Customer Level                                                       1   
How likely are you to recommend insider to a fr...                2013   
How would you rate the value you gain from our ...                2013   
Please rate the overall quality of our products                   2013   
Please rate the usability of the panel                            2018   
MRR                                                               1319   
Total Revenue                                                     1750   
Help Ticket Count

In [56]:
merged_df["Company Newsletter Interaction Count"] = merged_df["Company Newsletter Interaction Count"].fillna(0)
merged_df["Product Bug Task Count"] = merged_df["Product Bug Task Count"].fillna(0)
csat_columns = [
    "How likely are you to recommend insider to a friend or colleague ",
    "How would you rate the value you gain from our company",
    "Please rate the overall quality of our products",
    "Please rate the usability of the panel"
]

numeric_cols = merged_df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    merged_df[col] = merged_df[col].fillna(merged_df[col].median())
merged_df["MRR"] = merged_df["MRR"].fillna(0)
merged_df["Total Revenue"] = merged_df["Total Revenue"].fillna(0)
merged_df["Help Ticket Count"] =merged_df["Help Ticket Count"].fillna(0)
merged_df["Help Ticket Lead Time (hours)"]=merged_df["Help Ticket Lead Time (hours)"].fillna(0)
categorical_columns = ["Region", "Vertical", "Subvertical", "Customer Level"]

for col in categorical_columns:
    merged_df[col] = merged_df[col].fillna(merged_df[col].mode()[0])  

In [57]:
print(merged_df.isnull().sum())

Customer ID                                                          0
Customer Age (Months)                                                0
Company Newsletter Interaction Count                                 0
Product Bug Task Count                                               0
Region                                                               0
Vertical                                                             0
Subvertical                                                          0
Status                                                               0
Customer Level                                                       0
How likely are you to recommend insider to a friend or colleague     0
How would you rate the value you gain from our company               0
Please rate the overall quality of our products                      0
Please rate the usability of the panel                               0
MRR                                                                  0
Total 